In [1]:
# -*- coding: utf-8 -*-

In [2]:
# add path
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project1'
sys.path.insert(0,my_path + r'\code\COMMON')

# import external modules
import numpy as np
import matplotlib.pyplot as plt

# import internal modules
from proj1_helpers import load_csv_data 
from implementations import *

# Load Data

In [5]:
y, input_data, ids = load_csv_data(my_path + r'\data\train.csv', sub_sample=False)

In [6]:
np.shape(input_data)

(250000, 30)

# Handle the -999

# Separate PRI and DER in the input data

In [11]:
# names of the features 
featureNames = np.genfromtxt(my_path + r'\data\train.csv', delimiter=",", dtype=str, max_rows = 1)[2:]
print(featureNames)

# derived features from primitive data
x_der = X_no_999[:,:13]

# primitive data
x_pri = X_no_999[:,13:]

print("all: ", X_no_999.shape)
print("der: ",x_der.shape)
print("pri: ",x_pri.shape)
print("labels: ",y_no_999.shape)

['DER_mass_MMC' 'DER_mass_transverse_met_lep' 'DER_mass_vis' 'DER_pt_h'
 'DER_deltaeta_jet_jet' 'DER_mass_jet_jet' 'DER_prodeta_jet_jet'
 'DER_deltar_tau_lep' 'DER_pt_tot' 'DER_sum_pt' 'DER_pt_ratio_lep_tau'
 'DER_met_phi_centrality' 'DER_lep_eta_centrality' 'PRI_tau_pt'
 'PRI_tau_eta' 'PRI_tau_phi' 'PRI_lep_pt' 'PRI_lep_eta' 'PRI_lep_phi'
 'PRI_met' 'PRI_met_phi' 'PRI_met_sumet' 'PRI_jet_num' 'PRI_jet_leading_pt'
 'PRI_jet_leading_eta' 'PRI_jet_leading_phi' 'PRI_jet_subleading_pt'
 'PRI_jet_subleading_eta' 'PRI_jet_subleading_phi' 'PRI_jet_all_pt']
all:  (68109, 30)
der:  (68109, 13)
pri:  (68109, 17)
labels:  (68109,)


In [3]:
# PRI and DER 
initial_w = np.zeros(X_no_999.shape[1])
max_iters = 500
gamma = 0.000001
#w_ls, loss_ls = least_squares_GD(y_no_999, X_no_999, initial_w, max_iters, gamma)

# DER
initial_w = np.zeros(x_der.shape[1])
max_iters = 500
gamma = 0.00001
w_ls, loss_ls = least_squares_GD(y_no_999, x_der, initial_w, max_iters, gamma)

NameError: name 'X_no_999' is not defined

# Least Square SGD

In [12]:
# PRI and DER 
initial_w = np.zeros(X_no_999.shape[1])
max_iters = 500
gamma = 0.000001
w_ls, loss_ls = least_squares_SGD(y_no_999, X_no_999, initial_w, max_iters, gamma)

# DER
initial_w = np.zeros(x_der.shape[1])
max_iters = 500
gamma = 0.00001

w_ls, loss_ls = least_squares_SGD(y_no_999, x_der, initial_w, max_iters, gamma)

Gradient Descent(499/499): loss MSE=0.6153793802002161, w0=0.0013108777327664996, w1=0.0006597837931057937
Gradient Descent(499/499): loss MSE=4.4634768827650564e+154, w0=-2.6730853380421686e+75, w1=-9.116684127863998e+74


In [ ]:
# fixed hyper-parameters
degree = 5
lambda_min = -4
lambda_max = 10
k_fold = 4

# optimize the value of lambda given its range and the fixed hyper-parameters
lambda_opt = cross_validation_lambda(degree, lambda_min, lambda_max, k_fold)

print("Optimal lambda: ", lambda_opt)